In [1]:
import torch

# import qpth.solvers.dynamic.solve as dynamic_solver
from qpth.util import get_sizes, extract_nBatch, expandParam
from qpth.solvers import cvxpy

In [2]:
def forward(Q, p, G, h, A, b, verbose=0, maxIter=100, dt=0.2):
    nineq, nz, neq, nBatch = get_sizes(G, A)

    # Base inverses and transposes
    Q_I = torch.inverse(Q)
    G_T = torch.transpose(G, -1, 1)
    A_T = torch.transpose(A, -1, 1)

    # Intermediate matrix expressions
    GQ_I = - G.bmm(Q_I)             # - G Q^{-1}
    AQ_I = - A.bmm(Q_I)             # - A Q^{-1}
    GA = GQ_I.bmm(A_T)              # - G Q^{-1} A^T
    GG = GQ_I.bmm(G_T)              # - G Q^{-1} G^T
    AA = AQ_I.bmm(A_T)              # - A Q^{-1} A^T
    AG = AQ_I.bmm(G_T)              # - A Q^{-1} G^T
    la_d = GQ_I.bmm(p.unsqueeze(2)) - h.unsqueeze(2)       # - G Q^{-1} p - h
    nu_d = AQ_I.bmm(p.unsqueeze(2)) - b.unsqueeze(2)       # - A Q^{-1} p - b

    lams = torch.zeros(nBatch, nineq, 1).type_as(Q).to(Q.device)
    nus = torch.zeros(nBatch, neq, 1).type_as(Q).to(Q.device)
    zeros = torch.zeros(nBatch, nineq, 1).type_as(Q).to(Q.device)

    for _ in range(maxIter):
        dlams = dt * (GG.bmm(lams) + GA.bmm(nus) + la_d)
        dnus = dt * (AG.bmm(lams) + AA.bmm(nus) + nu_d)
        dlams = torch.max(lams + dlams, zeros) - lams
        lams.add_(dlams)
        nus.add_(dnus)

    zhat = - Q_I.bmm(p.unsqueeze(2) + G_T.bmm(lams) + A_T.bmm(nus))
    slacks = h.unsqueeze(2) - G.bmm(zhat)
    
    return zhat.squeeze(2), lams.squeeze(2), nus.squeeze(2), slacks.squeeze(2)

def forward_ineq(H, c, A, b, verbose=0, maxIter=100, dt=0.2):
    nineq, nz, neq, nBatch = get_sizes(A)
    A_T = torch.transpose(A, -1,1)
    H_I = torch.inverse(H)

    AH_I = A.bmm(H_I)
    D = - AH_I.bmm(A_T)
    d = - (b.unsqueeze(2) + AH_I.bmm(c.unsqueeze(2)))

    y = torch.zeros(nBatch, nineq, 1).type_as(H).to(H.device)
    zeros = torch.zeros(nBatch, nineq, 1).type_as(H).to(H.device)

    for _ in range(maxIter):
        dy = dt * (D.bmm(y) + d)
        dy = torch.max(y + dy, zeros) - y
        y.add_(dy)

    zhat = - H_I.bmm(c.unsqueeze(2) + A_T.bmm(y))
    slacks = b.unsqueeze(2) - A.bmm(zhat)
    
    return zhat.squeeze(2), y.squeeze(2), slacks.squeeze(2)

def forward_eq_conv(H, c, G_, h_, A_, b_, verbose=0, maxIter=100, dt=0.2):
    nineq, nz, neq, nBatch = get_sizes(G_, A_)

    A = torch.cat((G_, A_, -A_), dim=1)
    b = torch.cat((h_, b_, -b_), dim=1)
    A_T = torch.transpose(A, -1,1)
    H_I = torch.inverse(H)

    AH_I = A.bmm(H_I)
    D = - AH_I.bmm(A_T)
    d = - (b.unsqueeze(2) + AH_I.bmm(c.unsqueeze(2)))

    y = torch.zeros(nBatch, nineq + neq + neq, 1).type_as(H).to(H.device)
    zeros = torch.zeros(nBatch, nineq + neq + neq, 1).type_as(H).to(H.device)

    for _ in range(maxIter):
        dy = dt * (D.bmm(y) + d)
        dy = torch.max(y + dy, zeros) - y
        y.add_(dy)

    zhat = - H_I.bmm(c.unsqueeze(2) + A_T.bmm(y))
    slacks = b.unsqueeze(2) - A.bmm(zhat)
    
    return zhat.squeeze(2), y.squeeze(2)[:,:nineq], y.squeeze(2)[:,-neq:], slacks.squeeze(2)

def forward_cvxpy(Q_, p_, G_, h_, A_, b_):
    nBatch = extract_nBatch(Q_, p_, G_, h_, A_, b_)
    Q, _ = expandParam(Q_, nBatch, 3)
    p, _ = expandParam(p_, nBatch, 2)
    G, _ = expandParam(G_, nBatch, 3)
    h, _ = expandParam(h_, nBatch, 2)
    A, _ = expandParam(A_, nBatch, 3)
    b, _ = expandParam(b_, nBatch, 2)

    check_Q_spd = True
    if check_Q_spd:
        for i in range(nBatch):
            e, _ = torch.eig(Q[i])
            if not torch.all(e[:,0] > 0):
                raise RuntimeError('Q is not SPD.')

    _, nineq, nz = G.size()
    neq = A.size(1) if A.nelement() > 0 else 0
    assert(neq > 0 or nineq > 0)

    vals = torch.Tensor(nBatch).type_as(Q)
    zhats = torch.Tensor(nBatch, nz).type_as(Q)
    lams = torch.Tensor(nBatch, nineq).type_as(Q)
    nus = torch.Tensor(nBatch, neq).type_as(Q) \
        if neq > 0 else torch.Tensor()
    slacks = torch.Tensor(nBatch, nineq).type_as(Q)
    for i in range(nBatch):
        Ai, bi = (A[i], b[i]) if neq > 0 else (None, None)
        vals[i], zhati, nui, lami, si = cvxpy.forward_single_np(
            *[x.cpu().numpy() if x is not None else None
            for x in (Q[i], p[i], G[i], h[i], Ai, bi)])
        # if zhati[0] is None:
        #     import IPython, sys; IPython.embed(); sys.exit(-1)
        zhats[i] = torch.Tensor(zhati)
        lams[i] = torch.Tensor(lami)
        slacks[i] = torch.Tensor(si)
        if neq > 0:
            nus[i] = torch.Tensor(nui)

    return zhats, lams, nus, slacks


In [3]:
# Example data from: https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Quadratic_Programming.pdf

p = torch.tensor([1, -2, 4], dtype=torch.float).unsqueeze(0).cuda()
Q = torch.tensor([[2,0,1],[0,4,0],[1,0,6]], dtype=torch.float).unsqueeze(0).cuda()
h = torch.tensor([10, -2, 5, 5, 5, 0, 1, 0], dtype=torch.float).unsqueeze(0).cuda()
G = torch.tensor([
    [3, 4, -2],
    [2, -2, -1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, -1]
], dtype=torch.float).unsqueeze(0).cuda()
b = torch.tensor([5], dtype=torch.float).unsqueeze(0).cuda()
A = torch.tensor([
    [2, 3, 4]
], dtype=torch.float).unsqueeze(0).cuda()

In [10]:
# Standard Forward

%timeit forward(Q, p, G, h, A, b, maxIter=40)
zhat, lams, nus, slacks = forward(Q, p, G, h, A, b, maxIter=40)

print(zhat)
print(lams)
print(nus)
print(slacks)

58 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[0.0000, 0.1716, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[ 3.2789e+00, -3.5763e-07,  4.6053e+00,  3.6105e+00,  4.9895e+00,
          3.9474e-01,  2.3895e+00,  1.0526e-02]], device='cuda:0')


In [5]:
# Convert eq to ineq

%timeit forward_eq_conv(Q, p, G, h, A, b, maxIter=100)
zhat, lams, nus, slacks = forward_eq_conv(Q, p, G, h, A, b, maxIter=100)

print(zhat)
print(lams)
print(nus)
print(slacks)

77.8 ms ± 8.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
tensor([[-1.9730e+16, -1.4202e+16, -6.4037e+14]], device='cuda:0')
tensor([[7.0916e+15, 0.0000e+00, 1.3037e+15, 1.1712e+15, 1.2346e+15, 7.3825e+14,
         1.1874e+14, 0.0000e+00]], device='cuda:0')
tensor([[0.]], device='cuda:0')
tensor([[ 1.1472e+17,  1.0415e+16,  1.9730e+16,  1.4202e+16,  6.4037e+14,
         -1.9730e+16, -1.4202e+16, -6.4037e+14,  8.4629e+16, -8.4629e+16]],
       device='cuda:0')


In [7]:
# CVXPY

%timeit forward_cvxpy(Q, p, G, h, A, b)
zhat, lams, nus, slacks = forward_cvxpy(Q, p, G, h, A, b)

print(zhat)
print(lams)
print(nus)
print(slacks)

24.3 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
tensor([[0.3947, 1.3895, 0.0105]], device='cuda:0')
tensor([[ 0.0000e+00,  1.7158e-01,  0.0000e+00,  0.0000e+00, -1.6051e-15,
          1.0195e-21,  8.0256e-16, -1.1282e-21]], device='cuda:0')
tensor([[-1.0716]], device='cuda:0')
tensor([[3.2789e+00, 1.6856e-22, 4.6053e+00, 3.6105e+00, 4.9895e+00, 3.9474e-01,
         2.3895e+00, 1.0526e-02]], device='cuda:0')
